CLASES Y TIPADO

In [7]:
from typing import Dict, List, Set, Tuple, Callable


class Greetings:
    ConditionFuntion = Callable[[int]]
    def __init__(self, default_name) -> None:
        self.default_name = default_name
    def greet(self, name: str) -> str:
        return f"Hello, {name}"
    
    def printList(self, opciones: List[str]) -> str:
        return ", ".join(opciones)

    
c = Greetings("Alan")


print(c.default_name)  

print(c.greet("John")) 

print(c.printList(["Pedro", "Antonio"]))

Alan
Hello, John
Pedro, Antonio


Type function signatures with Callable

In [9]:
from typing import Callable, List

ConditionFunction = Callable[[int], bool]

def filter_list(l: List[int], condition: ConditionFunction) -> List[int]:
    return [i for i in l if condition(i)]


def is_even(i: int) -> bool:
    return i % 2 == 0

filter_list([1, 2, 3, 4, 5], is_even)

[2, 4]

In [2]:
class Temperature:
    def __init__(self, value, scale):
        self.value = value
        self.scale = scale
    def __repr__(self):
     return f"Temperature({self.value}, {self.scale!r})"
    def __str__(self):
      return f"Temperature is {self.value} °{self.scale}"
t = Temperature(25, "C")
print(repr(t))  # "Temperature(25, 'C')"
print(str(t))  # "Temperature is 25 °C"
print(t)

Temperature(25, 'C')
Temperature is 25 °C
Temperature is 25 °C


Any and cast

In [13]:
from typing import Any
#cualquiera
def f(x: Any) -> Any:
    return x

from typing import cast
def f(x: Any) -> Any:
    return x
a = f("a")  # inferred type is "Any"
a = cast(str, f("a"))  # forced type to be "str"


str

In [15]:
from fastapi import FastAPI, Query
app = FastAPI()
@app.get("/")
async def hello_world():
    return {"hello": "world"}

In [ ]:
#validacon de opciones

from enum import Enum
from fastapi import FastAPI
class UserType(str, Enum):
    STANDARD = "standard"
    ADMIN = "admin"
    
    

@app.get("/users/{type}/{id}/")
async def get_user(type: UserType, id: int):
    return {"type": type, "id": id} 

In [ ]:
@app.get("/license-plates/{license}")
async def get_license_plate(license: str = Path(..., regex=r"^\
w{2}-\d{3}-\w{2}$")):
    return {"license": license}

In [16]:
#parametros
@app.get("/users")
async def get_user(page: int = 1, size: int = 10):
    return {"page": page, "size": size}

In [ ]:
@app.get("/users")
async def get_user(page: int = Query(1, gt=0), size: int =
Query(10, le=100)):
    return {"page": page, "size": size}

In [193]:

import psycopg2
import pandas as pd
import json

param_dic = {
    "host"      : "localhost",
    "user"      : "postgres",
    "password"  : "admin",
    "port": 5433
}


def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    return conn
    
    
freq = {"Anual": "Y",
        "Mensual": "M",
        "Semanal": "W",
        "Diario": "D"
        }

def generacion_query( type: str, countries: list, options: list | str, frecuency: str):
     
    if type == "source":
        opcion_clase = 'sc.name'
    elif type == "category":
        opcion_clase = 'sc.type'

    countries = ", ".join(f"'{w}'" for w in countries)


    conn = connect(param_dic)
    cursor = conn.cursor()

    if isinstance(options, str):
        sql = f'''SELECT fact.timestamp_id, country.name_es, {opcion_clase}, SUM(fact.generation)
                    FROM factgeneration fact JOIN dimensionsources sc
                    ON fact.source_id = sc.id
                    JOIN dimensioncountry country ON fact.country_id = country.id
                    GROUP BY fact.timestamp_id, country.name_es, {opcion_clase}
                    HAVING country.name_es IN ({countries})'''
        
    else:
        options_str = ", ".join(f"'{w}'" for w in options)
        sql = f'''SELECT fact.timestamp_id, country.name_es, {opcion_clase}, SUM(fact.generation)
                    FROM factgeneration fact JOIN dimensionsources sc
                    ON fact.source_id = sc.id
                    JOIN dimensioncountry country ON fact.country_id = country.id
                    GROUP BY fact.timestamp_id, country.name_es, {opcion_clase}
                    HAVING country.name_es IN ({countries}) and {opcion_clase} IN ({options_str}) '''
    
    
    cursor.execute(sql)
    query = cursor.fetchall()

    df = pd.DataFrame(
        query, columns=["Timestamp", "Country", "Type", "Generation"])

    df["Timestamp"] = pd.to_datetime(df["Timestamp"], utc=True)
    df = (
        df.pivot_table(values=['Generation'], index="Timestamp", columns=[
                       'Type', 'Country'], aggfunc='first')
        .groupby(pd.Grouper(freq=freq[frecuency], level=0, closed='left'))
        .sum()
    )

    df.columns = df.columns.droplevel(0)
    df = (df.reset_index()
          .melt(id_vars="Timestamp", value_vars=None, var_name=["Type", "Country"],
                value_name='Generation', col_level=None, ignore_index=True)
          )

    
    return df



In [ ]:
df = (
        df.pivot_table(values=['Generation'], index="Timestamp", columns=[
                       'Type', 'Country'], aggfunc='first')
        .groupby(pd.Grouper(freq=freq[frecuency], level=0, closed='left'))
        .sum()
    )

    df.columns = df.columns.droplevel(0)

    df = (df.reset_index()
          .melt(id_vars="Timestamp", value_vars=None, var_name=["Type", "Country"],
                value_name='Generation', col_level=None, ignore_index=True)
          )



In [195]:
h=generacion_query(type="source", countries=["Francia", "Alemania"], options="all", frecuency="Mensual")
h

,Timestamp,Type,Country,Generation
0,2015-01-31 00:00:00+00:00,Biomass,Alemania,2963450.25
1,2015-02-28 00:00:00+00:00,Biomass,Alemania,2692398.25
2,2015-03-31 00:00:00+00:00,Biomass,Alemania,2931919.50
3,2015-04-30 00:00:00+00:00,Biomass,Alemania,2845559.75
4,2015-05-31 00:00:00+00:00,Biomass,Alemania,2918101.75
...,...,...,...,...
2683,2022-08-31 00:00:00+00:00,Wind Onshore,Francia,2098805.0
2684,2022-09-30 00:00:00+00:00,Wind Onshore,Francia,2275253.0
2685,2022-10-31 00:00:00+00:00,Wind Onshore,Francia,3412959.0
2686,2022-11-30 00:00:00+00:00,Wind Onshore,Francia,4697240.0


In [99]:
frecuency="Diario"
if frecuency == "Anual":
    h[h["Timestamp"].dt.year == h.Timestamp.max().year]
if frecuency == "Mensual":
    h[h["Timestamp"] >= h.Timestamp.max().strftime('%b-%Y')]
    
if frecuency == "Diario":
    h[h["Timestamp"] >= h.Timestamp.max().strftime('%d-%b-%Y')]

In [100]:
h[h["Timestamp"] >= h.Timestamp.max().strftime('%d-%b-%Y')]

,Timestamp,Type,Country,Generation
2905,2022-12-14 00:00:00+00:00,Biomass,Francia,6373.0
5811,2022-12-14 00:00:00+00:00,Fossil Gas,Francia,190220.0
8717,2022-12-14 00:00:00+00:00,Fossil Hard coal,Francia,35221.0
11623,2022-12-14 00:00:00+00:00,Fossil Oil,Francia,9097.0
14529,2022-12-14 00:00:00+00:00,Hydro Pumped Storage,Francia,19859.0
17435,2022-12-14 00:00:00+00:00,Hydro Run-of-river and poundage,Francia,104881.0
20341,2022-12-14 00:00:00+00:00,Hydro Water Reservoir,Francia,53933.0
23247,2022-12-14 00:00:00+00:00,Nuclear,Francia,872477.0
26153,2022-12-14 00:00:00+00:00,Solar,Francia,11983.0
29059,2022-12-14 00:00:00+00:00,Waste,Francia,2093.0


In [97]:
pd.to_datetime(h.Timestamp.max().strftime('%d-%b-%Y') )

Timestamp('2022-12-31 00:00:00')

In [57]:
def f1(func):
    def wrapper(*args, **kwargs):
        
        result= func(*args, **kwargs)
        return result + "fffg"
        
    return wrapper
        

In [114]:
pip install requests

  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 6.6 MB/s eta 0:00:00
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [156]:
import requests

def query_list(variable,  lista):
    query=""
    for elem in lista:
        query = query + variable + f"={elem}&"
    return query[:-1]

country=["Francia"]
url = "http://127.0.0.1:8000/items/source?frecuency=Anual&last=true&percentage=true&"
url = url + query_list("country", country)
res = requests.get(url = url)
df = pd.read_json(res.text, orient='records')
final = df[df["Generation"]==df.Generation.max()]

return final.Type.values[0],final.Generation.values[0]

,Timestamp,Type,Country,Generation
0,2022-12-31,Biomass,Francia,0.819864
1,2022-12-31,Fossil Gas,Francia,9.890396
2,2022-12-31,Fossil Hard coal,Francia,0.698913
3,2022-12-31,Fossil Oil,Francia,0.428786
4,2022-12-31,Hydro Pumped Storage,Francia,1.208807
5,2022-12-31,Hydro Run-of-river and poundage,Francia,7.633703
6,2022-12-31,Hydro Water Reservoir,Francia,2.277571
7,2022-12-31,Nuclear,Francia,64.259025
8,2022-12-31,Solar,Francia,4.298721
9,2022-12-31,Waste,Francia,0.333151


In [204]:
def having_construction(countries, sources):
    condition = "HAVING "
    if countries is not None:
        countries_str =  ", ".join(f"'{w}'" for w in countries)
        condition = condition + f"country.name_es IN ({countries_str})"
    if  sources is not None:
        sources_str =  ", ".join(f"'{w}'" for w in sources)
        condition = condition + f"country.name IN ({sources_str})"
        
    if countries is None and sources is None:
        condition = ""
    
    return condition
        
having_construction(None, None)

In [165]:
def last_max_value(country):
    

    url = f"http://127.0.0.1:8000/items/source?frecuency=Anual&last=true&percentage=true&country={country}"

    res = requests.get(url = url)
    df = pd.read_json(res.text, orient='records')
    final = df[df["Generation"]==df.Generation.max()]
    return final.Type.values[0],final.Generation.values[0]



last_max_value("Francia")

('Nuclear', 64.2590251763)

In [183]:

percentage_map={
    "Total": "true",
    "Porcentual": "false"
}

inputs= {"percentage": "Total",
                                                      
                                                      "option": "Fossil Gas",
                                                      "option_freq": "Mensual",
                                                      "country": ["Francia", "Alemania"]}

percentage= percentage_map[inputs["percentage"]]
countries = query_list("country", inputs["country"])
sources = inputs["option"]
url=f"http://127.0.0.1:8000/items/source?percentage={percentage}&source={sources}&frecuency=Mensual&{countries}"
res = requests.get(url = url)
df = pd.read_json(res.text, orient='records')
df

,Timestamp,Type,Country,Generation
0,2015-01-31,Fossil Gas,Alemania,1.321740
1,2015-02-28,Fossil Gas,Alemania,1.671668
2,2015-03-31,Fossil Gas,Alemania,1.384670
3,2015-04-30,Fossil Gas,Alemania,1.014256
4,2015-05-31,Fossil Gas,Alemania,1.082828
...,...,...,...,...
187,2022-08-31,Fossil Gas,Francia,4.120843
188,2022-09-30,Fossil Gas,Francia,4.414192
189,2022-10-31,Fossil Gas,Francia,4.892776
190,2022-11-30,Fossil Gas,Francia,5.219374


In [182]:
url

"http://127.0.0.1:8000/items/source?percentage=true&source=['Fossil Gas']&frecuency=Mensual&country=Francia&country=Alemania"